## 1. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты). 

In [1]:
import re
import requests
from hashlib import md5
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
from pprint import pprint
from bs4 import BeautifulSoup as bs

### Функция для создания DOM из необходимой html страницы

In [2]:
def get_dom(vacancy, page=0):
    url_base = 'https://hh.ru/vacancies/'
    
    if vacancy is None:
        vacancy = input('Введите желаемую вакансию: ')

    url = url_base + vacancy

    params = {'page': str(page)}

    headers = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.174 YaBrowser/22.1.3.907 (beta) Yowser/2.5 Safari/537.36'}

    response = requests.get(
        url,
        params=params,
        headers=headers
    )

    dom = bs(response.text, 'html.parser')
    
    return dom

### Отдельно создадим функцию, генерирующюю хэш на основе вводимой информации, в эту функцию мы будем передавать список-документ
### Возвращаемый хэш используем в качестве id этого документа, таким образом мы исключим дубликаты в нашей базе данных

In [3]:
def hash_id(doc):
    bytes_input = str(doc).encode('utf-8')
    
    return md5(bytes_input).hexdigest()

### Функция для добавления новой информации, взятой из DOM

In [4]:
def load_vacancies(dom, mongo_collection, duplicate_info=False, website='https://hh.ru'):
    
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item-body__main-info'})
    if vacancies == []:
        if duplicate_info:
            return (0, 0)
        else:
            return 0
    
    num_added = 0
    duplicates = 0
    
    for vacancy in vacancies:
        new_vacancy = {}
        
        # Добавление поля name
        name = vacancy.find('a').getText()
        new_vacancy['name'] = name
        
        # Добавление полей min_salary и max_salary
        salary = vacancy.find('span', {'class': 'bloko-header-section-3'})
        if salary:
            salary = salary.getText().replace('\u202f', '')
        
        # Среди этих записей часто используется длинное тире, которого нет на обычной раскладке клавиатуры
        # Его номер в таблице ascii 8211б, потому я, также, добавил его сюда
        if salary is None:
            min_salary = None
            max_salary = None
        elif ('от' in salary and 'до' in salary) or '-' in salary or chr(8211) in salary:
            min_salary = int( re.search(r'\d+', salary)[0] )
            max_salary = int( re.findall(r'\d+', salary)[1] )
        elif 'от' in salary:
            min_salary = int( re.search(r'\d+', salary)[0] )
            max_salary = None
        elif 'до' in salary:
            min_salary = None
            max_salary = int( re.search(r'\d+', salary)[0] )
        
        new_vacancy['min_salary'] = min_salary
        new_vacancy['max_salary'] = max_salary
        
        # Добавление поля currency
        if salary:
            currency = re.search(r'[A-zА-я]{3}', salary)[0]
        else:
            currency = None
        
        new_vacancy['currency'] = currency
        
        # Добавление поля link
        link = vacancy.find('a')['href']
        new_vacancy['link'] = link
        
        # Добавление поля website
        new_vacancy['website'] = website
        
        # Добавление поля _id
        _id = hash_id(new_vacancy)
        new_vacancy['_id'] = _id
        
        # Добавление документа в базу данных
        try:
            mongo_collection.insert_one(new_vacancy)
            num_added += 1
        except DuplicateKeyError:
            duplicates += 1
    
    if duplicate_info:
        return (num_added, duplicates)
    else:
        return num_added

### Что бы выбрать нужную вакансию, придется вписать ее в нужной строке
### Это нужно для создания коллекции, специально под эту вакансию
### Также нужно будет ввести в поле ввода точно такое же значение

In [5]:
vacancy = input('Введите желаемую вакансию на английском языке: ').replace(' ', '-')

client = MongoClient('127.0.0.1', 27017)
db = client['user']
collection = db.stroitel        # <--- Допишите вакансию здесь, к примеру "db.stroitel"

vac_added = 0
dup_finded = 0

for i in range(40):
    print(f'{i+1} страница')
    dom = get_dom(vacancy, i)
    vacancy_info = load_vacancies(dom, collection, duplicate_info=True)
    
    
    if vacancy_info[0] == 0 and vacancy_info[1] == 0:
        break
        
    vac_added += vacancy_info[0]
    dup_finded += vacancy_info[1]
    
    
print()
print(f'Вакансий добавлено: {vac_added}')
print(f'Дубликатов обнаружено: {dup_finded}')

Введите желаемую вакансию на английском языке: stroitel
1 страница
2 страница
3 страница
4 страница
5 страница
6 страница
7 страница
8 страница
9 страница
10 страница
11 страница

Вакансий добавлено: 76
Дубликатов обнаружено: 406


In [6]:
result = list(collection.find({}))

for i in range(3):
    pprint(result[i])
    print()

{'_id': '152e7e98097da3a3e1e1eeead768c86b',
 'currency': 'руб',
 'link': 'https://hh.ru/vacancy/54534809?query=%D1%81%D1%82%D1%80%D0%BE%D0%B8%D1%82%D0%B5%D0%BB%D1%8C',
 'max_salary': 100000,
 'min_salary': 90000,
 'name': 'Водитель / Разнорабочий-строитель',
 'website': 'https://hh.ru'}

{'_id': '8a5d6e5a7598022e5a0e23222d9398da',
 'currency': 'руб',
 'link': 'https://hh.ru/vacancy/54519579?query=%D1%81%D1%82%D1%80%D0%BE%D0%B8%D1%82%D0%B5%D0%BB%D1%8C',
 'max_salary': 130000,
 'min_salary': None,
 'name': 'Инженер-строитель',
 'website': 'https://hh.ru'}

{'_id': 'ad39e4c77ff9dfd1d163b3729b387d2c',
 'currency': 'руб',
 'link': 'https://hh.ru/vacancy/53562072?query=%D1%81%D1%82%D1%80%D0%BE%D0%B8%D1%82%D0%B5%D0%BB%D1%8C',
 'max_salary': 250000,
 'min_salary': 80000,
 'name': 'Инженер-строитель',
 'website': 'https://hh.ru'}



## 2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты).

### Сначала я использую проверку на минимальную зарплату, а потом на максимальную. Это нужно на случай, если в поле минимальной зарплаты будет None

In [13]:
salary = int(input('Введите желаемую заработную плату: '))

select = {
    '$or':
         [{'min_salary': {'$gt': salary}},
          {'max_salary': {'%gt': salary}}]
}

for vac in collection.find(select):
    print()
    pprint(vac)

Введите желаемую заработную плату: 100000

{'_id': 'dd629f62c12b1a363c55da8c7bc98eee',
 'currency': 'руб',
 'link': 'https://hh.ru/vacancy/53409278?query=%D1%81%D1%82%D1%80%D0%BE%D0%B8%D1%82%D0%B5%D0%BB%D1%8C',
 'max_salary': None,
 'min_salary': 104000,
 'name': 'Инженер-строитель/Ведущий специалист по техническому надзору',
 'website': 'https://hh.ru'}

{'_id': '028fde6317f841266223041e1faf6aae',
 'currency': 'руб',
 'link': 'https://hh.ru/vacancy/53618780?query=%D1%81%D1%82%D1%80%D0%BE%D0%B8%D1%82%D0%B5%D0%BB%D1%8C',
 'max_salary': 180000,
 'min_salary': 130000,
 'name': 'Монтажник НВК',
 'website': 'https://hh.ru'}

{'_id': 'f6c5afb76a556fe26f55e5507629e8c9',
 'currency': 'руб',
 'link': 'https://hh.ru/vacancy/54500719?query=%D1%81%D1%82%D1%80%D0%BE%D0%B8%D1%82%D0%B5%D0%BB%D1%8C',
 'max_salary': None,
 'min_salary': 150000,
 'name': 'Начальник участка',
 'website': 'https://hh.ru'}

{'_id': '62c7ad9ea45fd7ede32db102b256c294',
 'currency': 'руб',
 'link': 'https://hh.ru/vacancy/5450